In [112]:
# %% imports and setting up
from os import chdir as setwd 
from os import getcwd as getcwd

# setting working directory
from config import file_dir
setwd(file_dir)
print("Current working directory: ", getcwd())

# import data science libraries and scripts
import pandas as pd
import numpy as np
import csv
import gurobipy as gp
from gurobipy import GRB

Current working directory:  /Users/jasonnathaniel/Documents/Monash University/Sem 2 2022/FIT3164 Data science project 2/ds08_renewable_energy_scheduling


## Import datasets for input

In [113]:
# forecast values
forecast = []
building_consumption = []
solar_production = []

# sample output from the 
with open('data/sample_output.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        forecast.append(row)
        
for i in range(0,6):
    building_consumption.append(forecast[i])
    
for i in range(6,12):
    solar_production.append(forecast[i])
    
# netload[i,t] = building consumption[i,t] - solar production[i,t] 
# where i is building/solar number and t is time 
netload = []

for t in range(1,2881):
    sum_building_consumption = 0
    sum_solar_production = 0
    for i in range(6):
        sum_building_consumption += float(building_consumption[i][t])
        sum_solar_production += float(solar_production[i][t])
    netload.append(sum_building_consumption-sum_solar_production)
        

In [114]:
# electricity price data
electricity = pd.read_csv('rawdata/PRICE_AND_DEMAND_202011_VIC1.csv')
electricity = electricity.loc[electricity.index.repeat(2)]
electricity = electricity.reset_index()

price_elec = electricity['RRP']

In [115]:
# crate day of the week for each time stamp in November 2020
# starts with Sunday followed by 4 weeks, and ends on Monday
e = []
for i in range(1,8):
    e += [i for _ in range(96)]

dayofweek = [7 for _ in range(96)] + e*4 + [1 for _ in range(96)]

In [116]:
# create time of day
# consist of 96 t(s), start with index 0 ends with 95

t = [i for i in range(96)]

In [117]:
optimData = {'Base_load' : netload ,
            'price' : price_elec,
            'dayofweek' : dayofweek,
            'timeofday' : t*30
            }
optimData = pd.DataFrame(optimData)
base_load = netload
optimData


,Base_load,price,dayofweek,timeofday
0,852.0,75.16,7,0
1,809.0,75.16,7,1
2,814.0,69.14,7,2
3,822.0,69.14,7,3
4,871.0,52.61,7,4
...,...,...,...,...
2875,793.0,54.46,1,91
2876,853.0,52.14,1,92
2877,811.0,52.14,1,93
2878,929.0,44.00,1,94


## Read problem instance

In [118]:
with open('rawdata/phase2_instance_small_1.txt') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    
lines = [line.split() for line in lines]
lines

# first line of problem instance values
ppoi = lines[0]
building_count = ppoi[1]
solar_count = ppoi[2]
battery_count = ppoi[3]
reccur_act_count = ppoi[4]
once_act_count = ppoi[5]

# get building data
building_instance = lines[1:1+int(building_count)]
names = ['instance_type','building_id','no_small_rooms','no_large_rooms']
building = pd.DataFrame(building_instance, columns = names)

# get solar data
solar_instance = lines[1+int(building_count):1+int(building_count)+int(solar_count)]
names = ['instance_type','solar_id','building_id']
solar = pd.DataFrame(solar_instance, columns = names)

# get battery data 
battery_instance = []
for i in range(len(lines)) : 
    if lines[i][0] == 'c' :
        battery_instance.append(lines[i])

if len(battery_instance) > 0 : 
    names = ['instance_type','battery_id','building_id','capacity_kwh','max_power_kwh','efficiency']
    battery_instance = pd.DataFrame(battery_instance, columns = names)
    
battery = battery_instance

# get recurring activity data
recurring_activity_instance = []

for i in range(len(lines)) : 
    if lines[i][0] == 'r' :
        if lines[i][6] == '0':
            recurring_activity_instance.append(lines[i].copy() + [[]])
        else : 
            recurring_activity_instance.append(lines[i][:7].copy() + [lines[i][7:].copy()])

for i in range(len(recurring_activity_instance)): 
    names = ['instance_type','activity_id','no_of_rooms','room_size','load_kwh','duration','no_of_precendence','precendence_list']
    recurring_activity_instance = pd.DataFrame(recurring_activity_instance, columns = names)
    
recurring_activity_ins = recurring_activity_instance

# get once-off activity data
once_activity_instance = []

for i in range(len(lines)) : 
    if lines[i][0] == 'a' :
        if lines[i][8] == '0':
            once_activity_instance.append(lines[i].copy() + [[]])
        else : 
            once_activity_instance.append(lines[i][:9].copy() + [lines[i][9:].copy()])

for i in range(len(recurring_activity_instance)): 
    names = ['instance_type','activity_id','no_of_rooms','room_size','load_kwh','duration','value','penalty','no_of_precendence','precendence_list']
    once_activity_instance = pd.DataFrame(once_activity_instance, columns = names)
    
once_activity_ins = once_activity_instance

In [119]:
building.iloc[:,1:] = building.iloc[:,1:].astype(int)
building

,instance_type,building_id,no_small_rooms,no_large_rooms
0,b,0,0,1
1,b,1,1,0
2,b,3,0,1
3,b,4,0,2
4,b,5,1,0
5,b,6,8,2


In [120]:
solar.iloc[:,1:] = solar.iloc[:,1:].astype(int)
solar

,instance_type,solar_id,building_id
0,s,0,0
1,s,1,1
2,s,2,3
3,s,3,4
4,s,4,5
5,s,5,6


In [121]:
battery.iloc[:,1:] = battery.iloc[:,1:].astype(float)
battery

,instance_type,battery_id,building_id,capacity_kwh,max_power_kwh,efficiency
0,c,0.0,1.0,150.0,75.0,0.85
1,c,1.0,3.0,420.0,60.0,0.6


In [122]:
recurring_activity_ins.iloc[:,[1,2,4,5,6]] = recurring_activity_ins.iloc[:,[1,2,4,5,6]].astype(int)

x = recurring_activity_ins.iloc[:,3]
small_room = [0]*len(x)
large_room = [0]*len(x)
for i in range(len(x)):
    if x[i] == "S":
        small_room[i] = recurring_activity_ins.iloc[:,2][i]
    else: 
        large_room[i] = recurring_activity_ins.iloc[:,2][i]
recurring_activity_ins['no_small_rooms'] = small_room
recurring_activity_ins['no_large_rooms'] = large_room
recurring_activity_ins.head()

,instance_type,activity_id,no_of_rooms,room_size,load_kwh,duration,no_of_precendence,precendence_list,no_small_rooms,no_large_rooms
0,r,0,1,S,143,8,9,"[5, 8, 16, 17, 24, 32, 41, 45, 49]",1,0
1,r,1,3,S,167,4,2,"[13, 37]",3,0
2,r,2,1,S,199,9,1,[1],1,0
3,r,3,3,L,142,4,0,[],0,3
4,r,4,2,S,151,5,8,"[1, 8, 11, 12, 32, 38, 42, 49]",2,0


In [123]:
once_activity_ins

,instance_type,activity_id,no_of_rooms,room_size,load_kwh,duration,value,penalty,no_of_precendence,precendence_list
0,a,0,1,S,120,2,11,18,1,[9]
1,a,1,1,S,164,6,41,59,6,"[0, 2, 7, 8, 14, 18]"
2,a,2,3,S,204,6,109,125,3,"[8, 11, 14]"
3,a,3,3,L,150,2,45,69,7,"[6, 7, 8, 9, 14, 15, 18]"
4,a,4,3,S,145,6,96,130,5,"[6, 7, 8, 11, 18]"
5,a,5,2,S,144,9,89,122,6,"[0, 1, 6, 8, 14, 18]"
6,a,6,2,S,190,4,45,37,3,"[7, 8, 18]"
7,a,7,3,S,169,4,77,126,0,[]
8,a,8,1,S,217,4,36,34,1,[7]
9,a,9,3,S,198,8,199,180,5,"[6, 7, 8, 11, 18]"


In [124]:
type(once_activity_ins['precendence_list'][0][0])

str

## Optimisation model

In [125]:
nbulding = 6
nsolar = 6
nperiod = 2880
nbattery = battery_instance.shape[0]
nreccur = recurring_activity_ins.shape[0]
nonce = once_activity_ins.shape[0]
nsmallroom = sum(building['no_small_rooms'])
nlargeroom = sum(building['no_large_rooms'])

w = 672
s = 132
d = 96

# week1 1 0-4
valid_t = list(range(s,s+32)) + list(range(s+d,s+d+32)) + list(range(s+2*d,s+2*d+32)) + list(range(s+3*d,s+3*d+32)) + list(range(s+4*d,s+4*d+32))

# week 2 7-11
for i in range(7,12):
    valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 
    
# week 3 14-18
for i in range(14,19):
    valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 
    
# week 4 21-25
for i in range(21,26):
    valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 

# week 5 28
i = 28
valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 
    

nonvalid_t = set(list(range(nperiod))) - set(valid_t)
nonvalid_t = list(nonvalid_t)

# there are 5 weeks in Nov 2020 that contains weekdays
week1_t = list(range(d,d+w))
week2_t = list(range(d+w,d+w*2))
week3_t = list(range(d+w*2,d+w*3))
week4_t = list(range(d+w*3,d+w*4))
week5_t = list(range(d+w*4,2880))



In [150]:
m = gp.Model("MonashScheduling")
m.setParam('TimeLimit', 10*60)

## variables
net_power = m.addVars(nperiod, vtype = GRB.CONTINUOUS, lb = 0,  name = 'net power') # net power at each time t 
act_power = m.addVars(nperiod, vtype = GRB.CONTINUOUS, lb = 0, name = 'act power') # power from acts at each time t
bat_level = m.addVars(nbattery, nperiod, vtype = GRB.CONTINUOUS, lb = 0, name = 'bat level') # battery level status at each time t
ract_begin = m.addVars(nreccur, nperiod, vtype = GRB.BINARY, name = 'recurring activity begin') # 1 if activity begins 
ract_ongoing = m.addVars(nreccur, nperiod, vtype = GRB.BINARY, lb = 0, name = 'recurring activity ongoing') # 1 if activity is ongoing
bat_charge = m.addVars(nbattery, nperiod, vtype = GRB.BINARY, name = 'battery charge')
bat_discharge = m.addVars(nbattery, nperiod, vtype = GRB.BINARY, name = 'battery discharge')

maxpower = m.addVar(name = "max power") # max power


## constraints

# -- battery -- 

    
for bat in range(nbattery):
    
    # each battery at start of the month is full
    m.addConstr(bat_level[bat,0] == battery['capacity_kwh'][bat])
    
    # each battery has to be less than the capacity
    for t in range(nperiod):
        m.addConstr(bat_level[bat,t] <= battery['capacity_kwh'][bat])
    
# -- activity --

for act in range(nreccur):

    # each activity happend only once a week 
    m.addConstr(gp.quicksum(ract_begin[act,t] for t in week1_t) == 1)
    m.addConstr(gp.quicksum(ract_begin[act,t] for t in week2_t) == 1)
    m.addConstr(gp.quicksum(ract_begin[act,t] for t in week3_t) == 1)
    m.addConstr(gp.quicksum(ract_begin[act,t] for t in week4_t) == 1)
    #m.addConstr(gp.quicksum(ract_begin[act,t] for t in week5_t) == 1)
    
    # precendence activity must happend before 
    if len(recurring_activity_ins['precendence_list'][act]) > 0 : 
        prec_list = recurring_activity_ins['precendence_list'][act]
        for prec_act in prec_list: 
            prec_act = int(prec_act)
            m.addConstr(gp.quicksum(ract_begin[act, t] * t for t in week1_t) >= gp.quicksum(ract_begin[prec_act, t] * t for t in week1_t))

    # same time every week 
    for num in range(768, nperiod):
        m.addConstr(ract_begin[act, num] == ract_begin[act, num - 24*7*4])
        m.addConstr(ract_ongoing[act, num] == ract_ongoing[act, num - 24*7*4])
    
    # activity should last as long as their period 
    for t in range(nperiod-recurring_activity_ins['duration'][act]):
        for dur in range(recurring_activity_ins['duration'][act]):
            m.addConstr(ract_begin[act,t] <= ract_ongoing[act, t + dur])
        
    # activity is only between monday to friday 9am-5pm
    for t in nonvalid_t:
        m.addConstr(ract_ongoing[act,t] == 0)
        m.addConstr(ract_begin[act,t] == 0)
        

for t in range(nperiod):
    # no of small room use in time t for all acitivity is less than equal to number of small rooms in all buildings
    m.addConstr(gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['no_small_rooms'][act] for act in range(nreccur))
                <= nsmallroom )
    
    # no of large room use in time t for all acitivity is less than equal to number of large rooms in all buildings
    m.addConstr(gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['no_large_rooms'][act] for act in range(nreccur))
                <= nlargeroom )
            
            
# -- max power -- 
m.addConstr(maxpower == gp.max_(net_power[t] for t in range(nperiod)))


# -- power calculation --

# power from all activity that is happening at each time t 
for t in range(nperiod):
     m.addConstr(act_power[t] == gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['load_kwh'][act] 
                                            * (recurring_activity_ins['no_small_rooms'][act] + recurring_activity_ins['no_large_rooms'][act])
                                            for act in range(nreccur)) )

# -- battery level calculation -- 
for t in range(1,nperiod):
    for bat in range(nbattery):
       m.addConstr(bat_level[bat,t] == bat_level[bat,t-1] + (bat_charge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** -0.5) 
                    - (bat_discharge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** 0.5))
        

# -- net power calculation -- 
for t in range(nperiod):
    m.addConstr(net_power[t] == base_load[t] + act_power[t] + gp.quicksum(bat_charge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** -0.5 
                   - bat_discharge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** 0.5 for bat in range(nbattery)))
    
    
## objective

#m.setObjective(maxpower)
#m.setObjective(gp.quicksum(net_power[t]*price_elec[t] for t in range(nperiod)), GRB.MINIMIZE)
m.setObjective(gp.quicksum(((0.25)*net_power[t]*price_elec[t])/1000 for t in range(nperiod)) + 0.005*maxpower**2)
 

## optimize 
m.optimize()


Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 1236997 rows, 311041 columns and 2853870 nonzeros
Model fingerprint: 0x12fbb2f7
Model has 1 quadratic objective term
Model has 1 general constraint
Variable types: 11521 continuous, 299520 integer (299520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e-05, 7e-02]
  QObjective range [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]

Interrupt request received
Presolve removed 1220203 rows and 278749 columns
Presolve time: 1.84s
Presolved: 16794 rows, 32292 columns, 147026 nonzeros
Presolved model has 1 quadratic objective terms
Variable types: 5757 continuous, 26535 integer (26308 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   80697    3.0920361e+04   2.167203e+03  

## Gettting the result from model and change to output format

In [147]:
obj = m.getObjective()
print("objective",obj.getValue())

var = m.getVars()
recurring_activity_ongoing = []
recurring_activity_begin = []
battery_charge = []
for v in var:
    #print(v)
    if 'recurring activity ongoing' in v.VarName:
        recurring_activity_ongoing.append(v.X) 
    if 'recurring activity begin' in v.VarName:
        recurring_activity_begin.append(v.X)

objective 34875.682631028845


In [148]:
res = pd.DataFrame(np.reshape(recurring_activity_begin,(nreccur,nperiod)))
ac = res.iloc[1]

for i in range(len(ac)):
    if ac[i] != 0.0 : 
        print(i)

145
817
1489
2161
2833


In [149]:
res = pd.DataFrame(np.reshape(recurring_activity_ongoing,(nreccur,nperiod)))
ac = res.iloc[1]

for i in range(len(ac)):
    if ac[i] != 0.0 : 
        print(i)

145
146
147
148
817
818
819
820
1489
1490
1491
1492
2161
2162
2163
2164
2833
2834
2835
2836
